In [48]:
import numpy as np
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import rasterio as rio
import math

In [2]:
#samples = gpd.read_file('/Users/AuerPower/Metis/git/crop-classification/data/intermediate/samples.shp')

## Functions for calculating vegetation indices from spectral bands
[Source of formulas](https://www.sentinel-hub.com/develop/documentation/eo_products/Sentinel2EOproducts)

In [51]:
### Normalized Difference Vegetation Index  (NDVI)
def ndvi(B08, B04):
    index = (B08 - B04) / (B08 + B04)
    return index

# Green Normalized Difference Vegetation Index   (GNDVI)
def gndvi(B08, B03):
    index = (B08 - B03) / (B08 + B03)
    return index

# Modified Chlorophyll Absorption in Reflectance Index (MCARI)
# General formula: ((700nm - 670nm) - 0.2 * (700nm - 550nm)) * (700nm /670nm)
def mcari(B05, B04, B03):
    index = ((B05 - B04) - 0.2 * (B05 - B03)) * (B05 / B04)
    return index

# Chlorophyll Red-Edge
# General formula: ([760:800]/[690:720])^(-1)
def red_edge(B07, B05):
    #index = math.pow((B07 / B05), (-1.0))
    index = (B07 / B05)**-1
    return index

# NDRE (Normalized difference red edge index) = NIR - RE/NIR + RE
def nrde(B08, B05):
    index = (B08 - B05)/(B08 + B05)
    return index

# Normalized Difference 819/1600 NDII (NDII)
# General formula: (819nm-1600nm)/(819nm+1600nm)
def ndii(B08, B11):
    index = (B08 - B11) / (B08 + B11)
    return index

# MSI - Simple Ratio 1600/820 Moisture Stress Index (MSI)
def msi(B11, B08):
    index = B11 / B08
    return index

## Add Imagery and calculate vegetation indices

In [62]:
# Open Bands with Rasterio and save to .tiff file 
img_dir = "/Users/AuerPower/Metis/git/crop-classification/data/imagery/tile1/S2A_MSIL1C_20170620T082011_N0205_R121_T34JEP_20170620T084200.SAFE/GRANULE/L1C_T34JEP_A010414_20170620T084200/IMG_DATA/"
b1 = rio.open(img_dir + "T34JEP_20170620T082011_B01.jp2")
b2 = rio.open(img_dir + "T34JEP_20170620T082011_B02.jp2")
b3 = rio.open(img_dir + "T34JEP_20170620T082011_B03.jp2")
b4 = rio.open(img_dir + "T34JEP_20170620T082011_B04.jp2")
b5 = rio.open(img_dir + "T34JEP_20170620T082011_B05.jp2")
b6 = rio.open(img_dir + "T34JEP_20170620T082011_B06.jp2")
b7 = rio.open(img_dir + "T34JEP_20170620T082011_B07.jp2")
b8 = rio.open(img_dir + "T34JEP_20170620T082011_B08.jp2")
b8a = rio.open(img_dir + "T34JEP_20170620T082011_B8A.jp2")
b9 = rio.open(img_dir + "T34JEP_20170620T082011_B09.jp2")
b10 = rio.open(img_dir + "T34JEP_20170620T082011_B10.jp2")
b11 = rio.open(img_dir + "T34JEP_20170620T082011_B11.jp2")
b12 = rio.open(img_dir + "T34JEP_20170620T082011_B12.jp2")
img_meta = b4.profile

In [63]:
img_meta

{'driver': 'JP2OpenJPEG', 'dtype': 'uint16', 'nodata': None, 'width': 10980, 'height': 10980, 'count': 1, 'crs': CRS.from_epsg(32734), 'transform': Affine(10.0, 0.0, 499980.0,
       0.0, -10.0, 6900040.0), 'blockxsize': 1024, 'blockysize': 1024, 'tiled': True}

In [58]:
## Calculate veg indices
ndvi_arr = ndvi(b8.read(1), b4.read(1))
gndvi_arr = gndvi(b8.read(1), b3.read(1))
#mcari_arr = mcari(b5.read(1), b4.read(1), b3.read(1))
re_arr = red_edge(b7.read(1), b5.read(1))
nrde_arr = nrde(b8a.read(1), b5.read(1))
ndii_arr = ndii(b8a.read(1), b11.read(1))
msi_arr = msi(b11.read(1), b8a.read(1))

In [66]:
# change metadata profile to write Gtiff format, for 6 bands and float values instead of int
img_meta['count'] = 6
img_meta['dtype'] = 'float64'
img_meta['driver'] = 'Gtiff'
img_meta

{'driver': 'Gtiff', 'dtype': 'float64', 'nodata': None, 'width': 10980, 'height': 10980, 'count': 6, 'crs': CRS.from_epsg(32734), 'transform': Affine(10.0, 0.0, 499980.0,
       0.0, -10.0, 6900040.0), 'blockxsize': 1024, 'blockysize': 1024, 'tiled': True}

In [68]:
# Create stack and save file
with rio.open('/Users/AuerPower/Metis/git/crop-classification/data/intermediate/veg_indices.tiff','w',
              **img_meta) as veg_indices:
    veg_indices.write(ndvi_arr,1) 
    veg_indices.write(gndvi_arr,2) 
    veg_indices.write(re_arr,3)
    veg_indices.write(nrde_arr,4) 
    veg_indices.write(ndii_arr,5)
    veg_indices.write(msi_arr,6)
    veg_indices.close()